In [61]:
import pandas as pd
from datetime import datetime
import numpy as np

In [37]:
members = pd.read_csv('data/NSS_GreatCare_Members.csv')
adm = pd.read_csv('data/NSS_Hospital_inpatient_admissions.csv')
pcp = pd.read_csv('data/NSS_PrimaryCare_Office_Visits.csv')

C:\Users\dhoza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [38]:
adm

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,2,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,...,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,...,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,...,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,...,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0


In [39]:
#choosing columns needed
thirtyadm = adm[['Patient ID','Facility Provider ID','Encounter Start Date','Encounter End Date']]

In [40]:
##renaming columns to be one "word"
thirtyadm.columns = ['patient_id','facility_id','start_date','end_date']
thirtyadm

,patient_id,facility_id,start_date,end_date
0,DS1|10033,DS1|NPI|1306890389,2013-01-12,2013-01-14
1,DS1|10043,DS1|NPI|1902803315,2014-01-05,2014-01-12
2,DS1|10073,DS1|NPI|1902803315,2012-06-13,2012-06-21
3,DS1|10101,DS1|NPI|1427055839,2012-12-06,2012-12-09
4,DS1|10118,DS1|NPI|1992818256,2015-03-11,2015-03-14
...,...,...,...,...
6208,DS1|9979,DS1|NPI|1689772592,2015-02-06,2015-02-11
6209,DS1|9981,DS1|NPI|1427055839,2014-08-07,2014-08-10
6210,DS1|9994,DS1|NPI|1689608150,2015-11-21,2015-11-23
6211,DS1|9994,DS1|NPI|1689608150,2015-11-24,2015-11-27


In [78]:
thirtyadm['next_adm']=thirtyadm.sort_values(by=['start_date'],ascending=True).groupby(['patient_id'])['start_date'].shift(-1)

<ipython-input-78-20eec885cd01>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thirtyadm['next_adm']=thirtyadm.sort_values(by=['start_date'],ascending=True).groupby(['patient_id'])['start_date'].shift(-1)


In [80]:
thirtyadm[thirtyadm['patient_id']=='DS1|9994']

,patient_id,facility_id,start_date,end_date,next_adm
6210,DS1|9994,DS1|NPI|1689608150,2015-11-21,2015-11-23,2015-11-24
6211,DS1|9994,DS1|NPI|1689608150,2015-11-24,2015-11-27,NaT


In [81]:
thirtyadm['diff']=thirtyadm['next_adm'] - thirtyadm['end_date']
thirtyadm

<ipython-input-81-dfee87bacf4c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thirtyadm['diff']=thirtyadm['next_adm'] - thirtyadm['end_date']


,patient_id,facility_id,start_date,end_date,next_adm,diff
0,DS1|10033,DS1|NPI|1306890389,2013-01-12,2013-01-14,NaT,NaT
1,DS1|10043,DS1|NPI|1902803315,2014-01-05,2014-01-12,NaT,NaT
2,DS1|10073,DS1|NPI|1902803315,2012-06-13,2012-06-21,NaT,NaT
3,DS1|10101,DS1|NPI|1427055839,2012-12-06,2012-12-09,NaT,NaT
4,DS1|10118,DS1|NPI|1992818256,2015-03-11,2015-03-14,NaT,NaT
...,...,...,...,...,...,...
6208,DS1|9979,DS1|NPI|1689772592,2015-02-06,2015-02-11,NaT,NaT
6209,DS1|9981,DS1|NPI|1427055839,2014-08-07,2014-08-10,NaT,NaT
6210,DS1|9994,DS1|NPI|1689608150,2015-11-21,2015-11-23,2015-11-24,1 days
6211,DS1|9994,DS1|NPI|1689608150,2015-11-24,2015-11-27,NaT,NaT


In [82]:
pt_readmitted=thirtyadm.dropna()
pt_readmitted

,patient_id,facility_id,start_date,end_date,next_adm,diff
8,DS1|10132,DS1|NPI|1386741635,2013-03-30,2013-04-01,2013-08-14,135 days
18,DS1|10253,DS1|NPI|1396725677,2010-06-02,2010-06-06,2011-10-15,496 days
19,DS1|10253,DS1|NPI|1689772592,2010-04-02,2010-04-16,2010-06-02,47 days
20,DS1|10291,DS1|NPI|1821250762,2010-05-14,2010-05-15,2011-02-24,285 days
34,DS1|10451,DS1|NPI|1023010113,2014-04-24,2014-04-26,2014-05-23,27 days
...,...,...,...,...,...,...
6198,DS1|9935,DS1|NPI|1669456299,2011-01-22,2011-01-24,2011-01-25,1 days
6199,DS1|9935,DS1|NPI|1902803315,2011-01-25,2011-01-26,2011-01-26,0 days
6201,DS1|9943,DS1|NPI|1184722779,2013-10-01,2013-10-16,2013-10-25,9 days
6202,DS1|9943,DS1|NPI|1184722779,2013-10-25,2013-10-28,2014-10-28,365 days


In [100]:
#pt_readmitted[pt_readmitted['patient_id']=='DS1|6964']

,patient_id,facility_id,start_date,end_date,next_adm,diff
5899,DS1|6964,DS1|NPI|1184722779,2018-07-21,2018-11-18,2018-07-21,-120 days
5898,DS1|6964,DS1|NPI|0000000000,2018-07-21,2018-09-18,2018-11-23,66 days


In [87]:
pt_readmitted=pt_readmitted.sort_values('start_date')
pt_readmitted

,patient_id,facility_id,start_date,end_date,next_adm,diff
77,DS1|10734,DS1|NPI|1104982917,2009-11-11,2009-11-25,2010-04-25,151 days
3367,DS1|39675,DS1|NPI|1184722779,2009-11-24,2010-11-30,2011-01-17,48 days
1304,DS1|21571,DS1|NPI|1689608150,2009-11-27,2009-12-02,2010-02-20,80 days
2612,DS1|3294,DS1|NPI|1710065933,2009-11-28,2009-12-08,2010-01-01,24 days
4982,DS1|53976,DS1|NPI|1689772592,2009-11-29,2009-12-01,2009-12-28,27 days
...,...,...,...,...,...,...
4642,DS1|51247,DS1|NPI|1184722779,2018-07-16,2018-07-21,2019-04-09,262 days
5899,DS1|6964,DS1|NPI|1184722779,2018-07-21,2018-11-18,2018-07-21,-120 days
5898,DS1|6964,DS1|NPI|0000000000,2018-07-21,2018-09-18,2018-11-23,66 days
922,DS1|18117,DS1|NPI|1033298617,2018-08-08,2018-08-11,2018-08-08,-3 days


In [102]:
pt_readmitted=pt_readmitted[pt_readmitted['diff']>'0 days']

                            

In [103]:
pt_readmitted

,patient_id,facility_id,start_date,end_date,next_adm,diff
77,DS1|10734,DS1|NPI|1104982917,2009-11-11,2009-11-25,2010-04-25,151 days
3367,DS1|39675,DS1|NPI|1184722779,2009-11-24,2010-11-30,2011-01-17,48 days
1304,DS1|21571,DS1|NPI|1689608150,2009-11-27,2009-12-02,2010-02-20,80 days
2612,DS1|3294,DS1|NPI|1710065933,2009-11-28,2009-12-08,2010-01-01,24 days
4982,DS1|53976,DS1|NPI|1689772592,2009-11-29,2009-12-01,2009-12-28,27 days
...,...,...,...,...,...,...
280,DS1|12364,DS1|NPI|1184722779,2018-04-04,2018-04-11,2018-05-02,21 days
3910,DS1|44744,DS1|NPI|1710918545,2018-05-01,2018-05-04,2018-06-12,39 days
4642,DS1|51247,DS1|NPI|1184722779,2018-07-16,2018-07-21,2019-04-09,262 days
5898,DS1|6964,DS1|NPI|0000000000,2018-07-21,2018-09-18,2018-11-23,66 days


In [95]:
pt_readmitted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 77 to 1334
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype          
---  ------       --------------  -----          
 0   patient_id   1512 non-null   object         
 1   facility_id  1512 non-null   object         
 2   start_date   1512 non-null   datetime64[ns] 
 3   end_date     1512 non-null   datetime64[ns] 
 4   next_adm     1512 non-null   datetime64[ns] 
 5   diff         1512 non-null   timedelta64[ns]
dtypes: datetime64[ns](3), object(2), timedelta64[ns](1)
memory usage: 82.7+ KB


In [104]:
pt_readm_thirty=pt_readmitted[pt_readmitted['diff']<= '30 days']
pt_readm_thirty

,patient_id,facility_id,start_date,end_date,next_adm,diff
2612,DS1|3294,DS1|NPI|1710065933,2009-11-28,2009-12-08,2010-01-01,24 days
4982,DS1|53976,DS1|NPI|1689772592,2009-11-29,2009-12-01,2009-12-28,27 days
3804,DS1|43565,DS1|NPI|1801152566,2009-12-09,2009-12-17,2009-12-30,13 days
3169,DS1|37954,DS1|NPI|1871543215,2009-12-23,2010-01-29,2010-02-16,18 days
2690,DS1|33763,DS1|NPI|1710918545,2009-12-23,2009-12-24,2010-01-21,28 days
...,...,...,...,...,...,...
2000,DS1|27178,DS1|NPI|1184722779,2017-08-16,2017-08-17,2017-08-28,11 days
1864,DS1|25963,DS1|NPI|1992818256,2017-09-10,2017-09-14,2017-09-15,1 days
3608,DS1|41795,DS1|NPI|1902803315,2017-12-25,2018-01-07,2018-01-31,24 days
279,DS1|12364,DS1|NPI|1184722779,2018-03-13,2018-03-18,2018-04-04,17 days


In [105]:
pt_readmitted.groupby(pd.Grouper(key='start_date',freq='BM')).count().dropna()


,patient_id,facility_id,end_date,next_adm,diff
start_date,,,,,
2009-11-30,6,6,6,6,6
2009-12-31,9,9,9,9,9
2010-01-29,8,8,8,8,8
2010-02-26,6,6,6,6,6
2010-03-31,5,5,5,5,5
...,...,...,...,...,...
2018-04-30,1,1,1,1,1
2018-05-31,1,1,1,1,1
2018-06-29,0,0,0,0,0


In [73]:
thirtyadm['start_date'] = pd.to_datetime(thirtyadm['start_date'], format='%Y-%m-%d')
thirtyadm['end_date'] = pd.to_datetime(thirtyadm['end_date'], format='%Y-%m-%d')

<ipython-input-73-01ffa3a95844>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thirtyadm['start_date'] = pd.to_datetime(thirtyadm['start_date'], format='%Y-%m-%d')
<ipython-input-73-01ffa3a95844>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thirtyadm['end_date'] = pd.to_datetime(thirtyadm['end_date'], format='%Y-%m-%d')


In [74]:
thirtyadm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   patient_id   6213 non-null   object        
 1   facility_id  6213 non-null   object        
 2   start_date   6213 non-null   datetime64[ns]
 3   end_date     6213 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 194.3+ KB


In [14]:
##getting denominator ---total admissions
totaladm = adm[adm.columns[0]].count()
totaladm

6213

In [ ]:
begin = '20150101'
>>> end = '20150228'
>>> list(date_range(begin, end, 4))